In [1]:
import os
import pandas as pd
import numpy as np

import re
import csv
from tqdm.auto import tqdm
import multiprocessing
import time

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
def clawringContentImage(href):
    response = requests.get(href)
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        
        res = soup.find('h3', {'id' : 'view_content'})
        content = res.text
        images = res.find_all('img', {'class' : 'lazy content-image'})

        if images != []:
            imageList = [url['data-original'] for url in res.find_all('img', {'class' : 'lazy content-image'})]
        else:
            imageList = []
        return content, imageList
    
    else:
        return np.nan, []    

In [3]:
def parsing_save(re):
    re = BeautifulSoup(re, 'html.parser')
    category = re.find('a', {'class' : 'mw_basic_list_category'}).text
    date = re.find('div', {'class' : 'mw_basic_list_datetime media-no-text'}).text
    view = re.find('div', {'class' : 'mw_basic_list_hit media-no-text'}).text
    title = re.find('h3', {'class' : 'media-list-subject flex'}).text.replace('\n', '')
    rank = str(re.find('img')['src']).split('/')[-1].split('.')[0]
    href = re.find_all('a')[1]['href']
    try:
        writer = re.find('a', {'class' : 'sv_member'}).text.strip()
    except:
        writer = re.find('div', {'mw_basic_list_name media-no-text'}).text
    content, imageList = clawringContentImage(href)

    with open(save_path, 'a', newline = '') as csvfile:
        csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
        
        try:
            csvWriter.writerow({'title' : title,
                                'category' : category,
                                'date' : date,
                                'writer' : writer,
                                'rank' : rank,
                                'view' : view,
                                'href' : href,
                                'content' : content,
                                'imageList' : imageList})      
        except:
            print(href, ': Error')

In [4]:
def clawringHref01(url, pageNum, startPage = 1):
    errorNum = 0

    if not os.path.isfile(save_path):
        tmp = pd.DataFrame(columns = fieldNames)
        tmp.to_csv(save_path, index = False)
        
        with open(save_path, 'w', newline = '') as csvfile:
            csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
            csvWriter.writeheader()

    for page in tqdm(range(startPage, pageNum + 1)):
        page = str(page)
        response = requests.get(url + page)
        if response.status_code == 200:
            html = response.text
            soup = BeautifulSoup(html, 'html.parser')
            re_list = soup.find_all('div', {"class":"list_wrap flex_spbtw"})            
            re_list = [str(re) for re in re_list]

            for re in re_list:
                parsing_save(re)

### 대다모 크롤링 대상
* 샴푸&두피케어
    * 샴푸&두피제품 찾기
    * 체험평가단
* 탈모톡톡
    * 탈모수다 : (완)
    * 샴푸&영양제 : (완)
* 탈모후기
    * 탈모치료성공스토리

1. 머리말 : category
2. 제목 : title
3. 작성자 : writer
4. 작성일 : date
5. 조회수 : view
6. 링크 : href
7. 계급 : rank

In [ ]:
path_save = 'href_탈모톡톡_탈모수다.csv'
path_save_base = '../Data'
save_path = os.path.join(path_save_base, path_save)
fieldNames = ['title', 'category', 'date', 'writer', 'rank', 'view', 'href', 'content', 'imageList']

#### 탈모톡톡_탈모수다

In [ ]:
url = 'https://daedamo.com/story?overlaps=1&page='
path_save = 'href_탈모톡톡_탈모수다.csv'
fieldNames = ['title', 'category', 'date', 'writer', 'rank', 'view', 'href', 'content', 'imageList']
pageNum = 2
startPage = 1
clawringHref01(url, pageNum, startPage)

#### 탈모톡톡_샴푸&영양제

In [ ]:
url = 'https://daedamo.com/balmo?overlaps=1&page='
path_save = 'href_탈모톡톡_샴푸&영양제.csv'
fieldNames = ['title', 'category', 'date', 'writer', 'rank', 'view', 'href', 'content', 'imageList']
pageNum = 120

clawringHref01(url, pageNum, path_save, fieldNames, startPage)

# 샴푸&두피케어-샴푸&두피케어
이거 같은 경우에는, 로그인을 해야하며, 

### selenium

In [15]:
home_url = 'https://daedamo.com/new/'
login_url = 'https://daedamo.com/new/bbs/login.php'

shamFind_url = 'https://daedamo.com/ingre?sca=%ED%83%88%EB%AA%A8%EA%B4%80%EB%A0%A8%EC%83%81%ED%92%88&bso=A&overlaps=5'
startPage = 1
endPage = 78

In [16]:
def crawReview(soup):
    commentList = soup.find_all('div', {'class', 'comment-item'})
    
    totalScoreList = []
    satisfactionScoreList = []
    priceScoreList = []
    rebuyScoreList = []
    commenterList = []
    commentDateList = []
    commentGoodList = []
    commentBadList = []
    commentContentList = []
    
    for comment in commentList:
        reviewScoreList = comment.find_all('div', {'class', 'review-item'})

        commenter = comment.find('span', {'class' : 'sv_wrap'})
        if pd.isnull(commenter):
            commenter = comment.find('span', {'class' : 'guest'}).text
        else:
            commenter = comment.find('a')['title'].replace(' 자기소개', '')
        
        commentDate = comment.find('span', {'class' : 'datetime'}).text  
        commentContent = comment.find('div', {'class' : 'contents'})
        
        
        commenterList.append(commenter)
        commentDateList.append(commentDate)
        
        
        if len(reviewScoreList) == 1:
            satisfactionScore = np.nan
            priceScore = np.nan
            rebuyScore = np.nan
            totalScore = reviewScoreList[0].find('span')['style'].replace('width:', '')[:-1]
            good = np.nan
            bad = np.nan
            commentContent = commentContent.text
                
        else:
            satisfactionScore = reviewScoreList[0].find('span')['style'].replace('width:', '')[:-1]
            priceScore = reviewScoreList[1].find('span')['style'].replace('width:', '')[:-1]
            rebuyScore = reviewScoreList[2].find('span')['style'].replace('width:', '')[:-1]
            totalScore = np.nan
            
            tmp_content = commentContent.find_all('dt')
            
            if len(tmp_content) == 2:
                commentContent = commentContent.text.split('아쉬운점')
                good = commentContent[0].split('좋은점')[1]
                bad = commentContent[1]
                
            else:
                good = commentContent.text.split('좋은점')[1]
                bad = np.nan
            commentContent = np.nan
            
        commentGoodList.append(good)
        commentBadList.append(bad)
        commentContentList.append(commentContent)      
        
        satisfactionScoreList.append(satisfactionScore)
        priceScoreList.append(priceScore)
        rebuyScoreList.append(rebuyScore)
        totalScoreList.append(totalScore)

    return [totalScoreList, satisfactionScoreList, priceScoreList, rebuyScoreList, commenterList, commentDateList, commentContentList, commentGoodList, commentBadList]

In [17]:
def saveResult(result1, resut2, savePath = '../Data/샴푸&두피제품 찾기.csv', fieldNames = ['href', 'title', 'reviewNum', 'tag', 'brand', 'company', 'howToUse', 'ingredients', 'image', 'volume', 'price', 'totalScore', 'satisfactionScore', 'priceScore', 'rebuyScore', 'commenter', 'commentDate', 'commentContent', 'commentGood', 'commentBad']):
        if result2 == []:
            with open(savePath, 'a', newline = '', encoding = 'utf-8-sig') as csvfile:
                csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
                csvWriter.writerow({'href' : result1[0],
                                    'title' : result1[1],
                                    'reviewNum' : result1[2],
                                    'tag' : result1[3],
                                    'brand' : result1[4],
                                    'company' : result1[5],
                                    'howToUse' : result1[6],
                                    'ingredients' : result1[7],
                                    'image' : result1[8],
                                    'volume' : result1[9],
                                    'price' : result1[10],
                                    'totalScore' : np.nan,
                                    'satisfactionScore' : np.nan,
                                    'priceScore' : np.nan,
                                    'rebuyScore' : np.nan,
                                    'commenter' : np.nan,
                                    'commentDate' : np.nan,
                                    'commentContent' : np.nan,
                                    'commentGood' : np.nan,
                                    'commentBad' : np.nan
                                   })
        else:
            tmp = [np.nan]*(len(result2[0]) -1)
            href = [result1[0]] + tmp
            title = [result1[1]] + tmp
            reviewNum = [result1[2]] + tmp
            tag = [result1[3]] + tmp
            brand = [result1[4]] + tmp
            company = [result1[5]] + tmp
            howToUse = [result1[6]] + tmp
            ingredients = [result1[7]] + tmp
            image = [result1[8]] + tmp
            volume = [result1[9]] + tmp
            price = [result1[10]] + tmp
            pd.DataFrame({'href' : href,
                          'title' : title,
                          'reviewNum' : reviewNum,
                          'tag' : tag,
                          'brand' : brand,
                          'company' : company,
                          'howToUse' : howToUse,
                          'ingredients' : ingredients,
                          'image' : image,
                          'volume' : volume,
                          'price' : price,
                          'totalScore' : result2[0],
                          'satisfactionScore' : result2[1],
                          'priceScore' : result2[2],
                          'rebuyScore' : result2[3],
                          'commenter' : result2[4],
                          'commentDate' : result2[5],
                          'commentContent' : result2[6],
                          'commentGood' : result2[7],
                          'commentBad' : result2[8],
                         }).to_csv(savePath, mode = 'a', index = False, header = None)      

In [19]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disabled-dev-shm-usage')
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
chrome_options.add_argument('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')

savePath = '../Data/샴푸&두피제품 찾기.csv'
fieldNames = ['href', 'title', 'reviewNum', 'tag', 'brand', 'company', 'howToUse', 'ingredients', \
             'image', 'volume', 'price', 'totalScore', 'satisfactionScore', 'priceScore', \
             'rebuyScore', 'commenter', 'commentDate', 'commentContent', 'commentGood', 'commentBad']

driver = webdriver.Chrome(options = chrome_options)

driver.get(login_url)
driver.implicitly_wait(np.random.rand(1)[0])

driver.find_element_by_id('login_id').send_keys('dataflow28')
driver.find_element_by_id('login_pw').send_keys('flowdata2833!!')
driver.find_element_by_class_name('btn_submit').click()
driver.implicitly_wait(np.random.rand(1)[0])
time.sleep(2)

numCompile = re.compile('\d')

if not os.path.isfile(savePath):
    tmp = pd.DataFrame(columns = fieldNames)
    tmp.to_csv(savePath, encoding = 'utf-8-sig')
    
    with open(savePath, 'w', newline = '', encoding = 'utf-8-sig') as csvfile:
        csvWriter = csv.DictWriter(csvfile, fieldnames = fieldNames)
        csvWriter.writeheader()

for page in tqdm(range(startPage, endPage + 1)):
    url = shamFind_url + '&page=' + str(page)
    
    driver.get(url)
    driver.implicitly_wait(np.random.rand(1)[0])
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    re_list = soup.find_all('div', {'class' : 'bbs_list_box flex_spbtw'})

    for res in re_list:
        href = res.find('h3', {'class' : 'cont_subject'}).find('a')['href']
        view = res.find('div', {'class' : 'cont_info_hit'}).text

        driver.get(href)
        driver.implicitly_wait(np.random.rand(1)[0])
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        content = soup.find_all('p', {'class' : 'cont'})
        howToUse = content[0].text
        ingredients = content[1].text
        try:
            image = soup.find('img', {'class' : 'lazy'})['src']
        except:
            image = np.nan
            
        tag = soup.find('div', {'class' : 'tag'}).text
        title = soup.find('div', {'class' : 'subject'}).text
        brand = soup.find('div', {'class' : 'brand'}).text
        company = soup.find('div', {'class' : 'company'}).text
        reviewNum = int(''.join(numCompile.findall(soup.find('div', {'class' : 'score_title'}).text)))
        
        maxPageNum = 0
        if tag == '탈모의료기기':
            volume = np.nan
            price = soup.find('span', {'class' : 'price'}).text
        else:
            try:
                volumeAndPrice = soup.find('div', {'class' : 'volume_price'}).text.strip().split('\n')
                volume = volumeAndPrice[0]
                price = volumeAndPrice[1]
            except:
                volume = np.nan
                price = np.nan
        
        result1 = [href, title, reviewNum, tag, brand, company, howToUse, ingredients, image, volume, price]
        
        if reviewNum == 0:
            result2 = []
            saveResult(result1, result2)
            continue
            
        elif reviewNum <= 20:
            result2 = crawReview(soup)
            saveResult(result1, result2)
            continue
        else:
            pageNextExist = True
            nextButtonClicked = False
            
            while pageNextExist:
                pageNext = soup.find('a', {'class' : 'pg_page pg_next'})
                if pd.isnull(pageNext):
                    pageNextExist = False
                result2 = crawReview(soup)
                saveResult(result1, result2)
                
                pgPageList = driver.find_elements(By.CLASS_NAME, 'pg_page')
                pgPageList_soup = soup.find_all('a', {'class' : 'pg_page'})
                pgPageList_soup = [int(pg['data-page_no']) for pg in pgPageList_soup]
                
                for i in range(len(pgPageList)):
                    if nextButtonClicked & (i == 0):
                        continue
                    
                    if pgPageList_soup[i] > maxPageNum:
                        maxPageNum = pgPageList_soup[i]                    
                    else :
                        pageNextExist = False
                        break
                    
                    pgPageList[i].click()
                    pgPageList = driver.find_elements(By.CLASS_NAME, 'pg_page')
                    driver.implicitly_wait(np.random.rand(1)[0])
                    time.sleep(1)
                    
                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    result2 = crawReview(soup)
                    saveResult(result1, result2)
                    
                    pageNext = soup.find('a', {'class' : 'pg_page pg_next'})
                    if pd.isnull(pageNext):
                        pageNextExist = False
                nextButtonClicked = True

  0%|          | 0/78 [00:00<?, ?it/s]